# セットアップ

①セットアップ

In [1]:
# 必要なライブラリのインストール
!pip install twikit pandas openpyxl

# 必要なインポート
import os
from google.colab import drive
from google.colab import files
import json
import pandas as pd
from datetime import datetime
from twikit import Client
import asyncio
from collections import Counter
from google.colab import output

# Google Driveのマウント
drive.mount('/content/drive')

# ディレクトリ設定
WORK_DIR = '/content/drive/MyDrive/Twitter_Analysis'
COOKIE_DIR = f"{WORK_DIR}/twitter_json"
RESULTS_DIR = f"{WORK_DIR}/profile_results"

# ディレクトリ作成
os.makedirs(COOKIE_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("セットアップ完了！")
print(f"Cookie path: {COOKIE_DIR}/cookie_edit.json")
print(f"Results directory: {RESULTS_DIR}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 3.2 MB/s eta 0:00:00
Mounted at /content/drive
セットアップ完了！
Cookie path: /content/drive/MyDrive/Twitter_Analysis/twitter_json/cookie_edit.json
Results directory: /content/drive/MyDrive/Twitter_Analysis/profile_results


②# クッキーファイルのアップロード X連携

# リプライサーチ

In [12]:
# クッキーファイルのアップロード
def upload_cookie_file():
    print("cookie_edit.jsonファイルをアップロードしてください")
    uploaded = files.upload()

    if 'cookie_edit.json' in uploaded:
        with open(f"{COOKIE_DIR}/cookie_edit.json", 'wb') as f:
            f.write(uploaded['cookie_edit.json'])
        print("クッキーファイルを保存しました")
        return True
    else:
        print("cookie_edit.jsonファイルがアップロードされませんでした")
        return False

upload_cookie_file()

cookie_edit.jsonファイルをアップロードしてください


Saving cookie_edit.json to cookie_edit (1).json
cookie_edit.jsonファイルがアップロードされませんでした


False

cookie_edit.jsonファイルをアップロードしてください


NameError: name 'files' is not defined

③初期実行　リプライサーチ

In [26]:

# TwitterProfileAnalyzerクラスの定義
class TwitterProfileAnalyzer:
    def __init__(self):
        self.client = Client(language='en-US')
        self.cookie_path = f"{COOKIE_DIR}/cookie_edit.json"
        self.results_dir = RESULTS_DIR

    async def setup(self):
        try:
            with open(self.cookie_path, 'r', encoding='utf-8') as file:
                cookies = json.load(file)
            self.client.set_cookies(cookies)
            print("認証に成功しました！")
            return True
        except Exception as e:
            print(f"認証エラー: {e}")
            return False

    async def analyze_user_replies(self, screen_name, tweets_to_analyze=200):
        """指定したユーザーのツイートから、リプライを分析する"""
        try:
            # ユーザー情報を取得
            target_user = await self.client.get_user_by_screen_name(screen_name)
            print(f"{screen_name}のツイートを分析中...")

            # リプライしているユーザーをスクリーンネームベースで追跡
            reply_counter = Counter()
            reply_users = {}

            # ツイートを取得（リプライを含む）
            print("ツイートを取得中...")
            results = await self.client.get_user_tweets(
                target_user.id,
                tweet_type='Replies',  # Repliesタイプに変更
                count=min(tweets_to_analyze, 100)
            )

            analyzed_count = 0
            while results and analyzed_count < tweets_to_analyze:
                for tweet in results:
                    try:
                        # リプライ先のツイートテキストを解析
                        if hasattr(tweet, 'text') and tweet.text.startswith('@'):
                            # @ユーザー名を抽出
                            mentioned_users = [
                                word[1:] for word in tweet.text.split()
                                if word.startswith('@')
                            ]

                            for reply_to in mentioned_users:
                                if reply_to and reply_to != screen_name:
                                    try:
                                        if reply_to not in reply_users:
                                            # スクリーンネームからユーザー情報を取得
                                            try:
                                                reply_user = await self.client.get_user_by_screen_name(reply_to)
                                                reply_users[reply_to] = reply_user
                                                print(f"ユーザー {reply_to} の情報を取得しました")
                                            except Exception as e:
                                                continue
                                        reply_counter[reply_to] += 1
                                    except Exception as e:
                                        print(f"ユーザー {reply_to} の情報取得をスキップ: {e}")
                                        continue
                    except Exception as e:
                        continue

                    analyzed_count += 1
                    if analyzed_count % 20 == 0:
                        print(f"{analyzed_count}件のツイートを分析済み")

                if analyzed_count < tweets_to_analyze and results.next_cursor:
                    try:
                        results = await results.next()
                    except Exception as e:
                        print(f"追加ツイート取得エラー: {e}")
                        break
                else:
                    break

            print(f"\n分析完了: {analyzed_count}件のツイートを処理")
            print(f"リプライ先ユーザー数: {len(reply_users)}人")

            return reply_counter, reply_users

        except Exception as e:
            print(f"分析エラー: {e}")
            return Counter(), {}

    async def get_user_profile(self, user):
        """ユーザーのプロフィール情報を取得する"""
        try:
            # プロフィール情報を辞書形式で整理
            profile_data = {
                'user_id': user.id,                      # ユーザーID
                'name': user.name,                       # 表示名
                'screen_name': user.screen_name,         # @ユーザー名
                'description': user.description,         # プロフィール文
                'location': user.location,               # 場所
                'followers_count': user.followers_count, # フォロワー数
                'following_count': user.following_count, # フォロー数
                'tweets_count': user.statuses_count,    # ツイート数
                'created_at': user.created_at,          # アカウント作成日
                'profile_image_url': user.profile_image_url  # プロフィール画像URL
            }
            return profile_data
        except Exception as e:
            print(f"プロフィール取得エラー: {e}")
            return None

    async def get_user_tweets(self, user, count=1):
        """ユーザーの投稿を取得する"""
        try:
            tweets = []
            results = await user.get_tweets(tweet_type='Tweets', count=count)

            for tweet in results:
                tweet_data = {
                    'tweet_id': tweet.id,                # ツイートID
                    'text': tweet.text,                  # ツイート本文
                    'created_at': tweet.created_at,      # 投稿日時
                    'retweet_count': tweet.retweet_count, # リツイート数
                    'like_count': tweet.favorite_count,   # いいね数
                    'reply_count': tweet.reply_count,     # 返信数
                    'is_retweet': bool(tweet.retweeted_tweet), # リツイートかどうか
                    'is_quote': tweet.is_quote_status     # 引用ツイートかどうか
                }
                tweets.append(tweet_data)

            return tweets
        except Exception as e:
            print(f"ツイート取得エラー: {e}")
            return []

    def save_results(self, frequent_repliers_data, target_screen_name):
        """分析結果をJSONファイルとして保存する"""
        try:
            current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = os.path.join(
                self.results_dir,
                f"analysis_{target_screen_name}_{current_time}.json"
            )

            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(frequent_repliers_data, f, ensure_ascii=False, indent=2)
            print(f"分析結果を保存しました: {filename}")
            return filename
        except Exception as e:
            print(f"保存エラー: {e}")
            return None

    def save_to_excel(self, frequent_repliers_data, target_screen_name):
        """分析結果をExcelファイルとして保存"""
        try:
            # データフレーム用のリストを作成
            rows = []
            for user_data in frequent_repliers_data:
                profile = user_data['profile']
                tweets = user_data['recent_tweets']

                # 最新のツイート3件を結合
                recent_tweets_text = "\n".join(
                    [tweet['text'] for tweet in tweets[:3]]
                ) if tweets else ""

                # 1行のデータとして整形
                row = {
                    '分析対象ユーザー': target_screen_name,
                    '分析日時': datetime.now().strftime("%Y-%m-%d %H:%M"),
                    'ユーザー名': profile['name'],
                    'ユーザーID': f"@{profile['screen_name']}",
                    'プロフィール文': profile['description'],
                    'アカウント作成日': profile['created_at'],
                    'リプライ数': user_data['reply_count'],
                    'フォロワー数': profile['followers_count'],
                    'フォロー数': profile['following_count'],
                    'ツイート数': profile['tweets_count'],
                    '場所': profile['location'],
                    'プロフィール画像URL': profile['profile_image_url'],
                    'アカウントURL': f"https://twitter.com/{profile['screen_name']}",
                    '最近のツイート': recent_tweets_text
                }
                rows.append(row)

            # DataFrameを作成
            df = pd.DataFrame(rows)

             # profile_resultsディレクトリを作成（存在しない場合）
            profile_results_dir = "profile_results"
            os.makedirs(profile_results_dir, exist_ok=True)

            # Excelファイル名を生成
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            excel_file = f"{self.results_dir}/リプライ分析_{target_screen_name}_{timestamp}.xlsx"

            # Excelファイルとして保存
            with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='リプライ分析結果', index=False)

                # 列幅の自動調整
                worksheet = writer.sheets['リプライ分析結果']
                for idx, col in enumerate(df.columns):
                    max_length = max(
                        df[col].astype(str).apply(len).max(),
                        len(str(col))
                    )
                    # 最大幅を50文字に制限
                    adjusted_width = min(max_length + 2, 50)
                    worksheet.column_dimensions[chr(65 + idx)].width = adjusted_width

                # 行の高さを調整（最近のツイート用）
                for row in range(2, len(df) + 2):  # Excelは1始まりでヘッダーがあるため+2
                    worksheet.row_dimensions[row].height = 60

            print(f"\nExcelファイルを保存しました: {excel_file}")
            return excel_file

        except Exception as e:
            print(f"Excelファイルの保存中にエラーが発生しました: {e}")
            return None


    async def run_analysis():
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            return

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # リプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: riku720720
最小リプライ数を入力してください（例: 3）: 3
分析するツイート数を入力してください（例: 200）: 200

riku720720のリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 3


/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:314: RuntimeWarning: coroutine 'run_analysis' was never awaited
  for attr in list(attrs.keys()):


riku720720のツイートを分析中...
ツイートを取得中...
20件のツイートを分析済み
ユーザー ai_syacho の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
100件のツイートを分析済み
ユーザー y_hatt の情報を取得しました
120件のツイートを分析済み
140件のツイートを分析済み
160件のツイートを分析済み
ユーザー jammaru_lab の情報を取得しました
ユーザー sora19ai の情報を取得しました
180件のツイートを分析済み
200件のツイートを分析済み

分析完了: 211件のツイートを処理
リプライ先ユーザー数: 4人

リプライの多いユーザーの情報を収集中...

分析対象となるユーザーが見つかりませんでした。


④2回目以降　実行用　リプライサーチ

In [27]:
async def run_analysis():
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            return

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # リプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 3
分析するツイート数を入力してください（例: 200）: 200

sora19aiのリプライを分析します...
- 分析対象ツイート数: 200
- 最小リプライ数: 3
sora19aiのツイートを分析中...
ツイートを取得中...
20件のツイートを分析済み
ユーザー 1108k4xi の情報を取得しました
ユーザー solt152152 の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
100件のツイートを分析済み
120件のツイートを分析済み
140件のツイートを分析済み
ユーザー sama の情報を取得しました
160件のツイートを分析済み
180件のツイートを分析済み
200件のツイートを分析済み
ユーザー 3_nipa の情報を取得しました

分析完了: 213件のツイートを処理
リプライ先ユーザー数: 4人

リプライの多いユーザーの情報を収集中...

分析対象となるユーザーが見つかりませんでした。


⑤2回目以降　実行用　リプライサーチ（X自動化のリプライ検索機能（相手から返信3回以上判定））

In [16]:
# TwitterプロフィールとリプライKPI分析用のクラス
class TwitterProfileAnalyzer:
    def __init__(self):
        """
        Twitterプロフィール分析クラスの初期化
        - クライアントとデータ保存用の変数を初期化
        """
        self.client = None
        self.COOKIE_PATH = f"{COOKIE_DIR}/cookie_edit.json"
        self.reply_analysis_data = {}

    async def setup(self) -> bool:
        """
        Twitter APIクライアントのセットアップと認証

        Returns:
            bool: セットアップ成功の場合True
        """
        try:
            # クライアントの初期化
            self.client = Client()

            # クッキーファイルが存在する場合は読み込み
            if os.path.exists(self.COOKIE_PATH):
                with open(self.COOKIE_PATH, 'r', encoding='utf-8') as f:
                    self.client.set_cookies(json.load(f))
                print("クッキー情報を読み込みました")
                return True
            else:
                print("クッキーファイルが見つかりません")
                return False
        except Exception as e:
            print(f"セットアップエラー: {e}")
            return False

    async def get_user_tweets(self, user, count=200):
        """
        指定ユーザーの最近のツイートを取得

        Args:
            user: ユーザーオブジェクト
            count: 取得するツイート数

        Returns:
            list: ツイートのリスト
        """
        try:
            tweets = await user.get_tweets('Tweets', count=count)
            return [tweet for tweet in tweets if hasattr(tweet, 'text')][:3]
        except Exception as e:
            print(f"ツイート取得エラー: {e}")
            return []

    async def analyze_user_replies(self, target_user: str, tweets_to_analyze: int) -> tuple:
        """
        ユーザーのリプライ分析を実行

        Args:
            target_user: 分析対象のユーザー名
            tweets_to_analyze: 分析対象のツイート数

        Returns:
            tuple: (リプライカウンター, リプライユーザー情報)
        """
        try:
            # ユーザー情報の取得
            user = await self.client.get_user_by_screen_name(target_user)
            if not user:
                print(f"ユーザー {target_user} が見つかりません")
                return Counter(), {}

            # ツイート取得と分析
            tweets = await user.get_tweets('Tweets', count=tweets_to_analyze)
            reply_counter = Counter()
            reply_users = {}
            replied_by_counter = Counter()  # 新機能: リプライされた回数のカウンター

            for tweet in tweets:
                if hasattr(tweet, 'reply_to') and tweet.reply_to:
                    for reply_tweet in tweet.reply_to:
                        if hasattr(reply_tweet.user, 'screen_name'):
                            screen_name = reply_tweet.user.screen_name
                            reply_counter[screen_name] += 1
                            reply_users[screen_name] = reply_tweet.user

                # 新機能: リプライされた回数の分析
                if hasattr(tweet, 'replies') and tweet.replies:
                    for reply in tweet.replies:
                        if hasattr(reply.user, 'screen_name'):
                            replied_by_counter[reply.user.screen_name] += 1

            # リプライ分析データの保存
            self.reply_analysis_data = {
                'outgoing_replies': dict(reply_counter),
                'incoming_replies': dict(replied_by_counter)
            }

            return reply_counter, reply_users

        except Exception as e:
            print(f"リプライ分析エラー: {e}")
            return Counter(), {}

    async def get_user_profile(self, user) -> dict:
        """
        ユーザープロフィール情報の取得と整形

        Args:
            user: ユーザーオブジェクト

        Returns:
            dict: プロフィール情報
        """
        try:
            return {
                'screen_name': user.screen_name,
                'name': user.name,
                'description': user.description,
                'followers_count': user.followers_count,
                'following_count': user.following_count,
                'tweets_count': user.statuses_count,
                'created_at': user.created_at,
                'location': user.location,
                'verified': user.verified,
                'profile_image_url': user.profile_image_url
            }
        except Exception as e:
            print(f"プロフィール情報取得エラー: {e}")
            return {}

    def save_results(self, results, target_user):
        """
        分析結果をJSONファイルとして保存

        Args:
            results: 分析結果データ
            target_user: 分析対象ユーザー名
        """
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{RESULTS_DIR}/{target_user}_analysis_{timestamp}.json"

        # 新機能: リプライ分析データを結果に追加
        save_data = {
            'user_profiles': results,
            'reply_analysis': self.reply_analysis_data
        }

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(save_data, f, ensure_ascii=False, indent=2)
            print(f"結果を保存しました: {filename}")
        except Exception as e:
            print(f"結果の保存に失敗: {e}")

    def save_to_excel(self, results, target_user):
        """
        分析結果をExcelファイルとして保存

        Args:
            results: 分析結果データ
            target_user: 分析対象ユーザー名
        """
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{RESULTS_DIR}/{target_user}_analysis_{timestamp}.xlsx"

        try:
            # プロフィール情報のDataFrame作成
            profiles_data = []
            for user_data in results:
                profile = user_data['profile']
                profile['reply_count'] = user_data['reply_count']
                # 新機能: 受信したリプライ数を追加
                screen_name = profile['screen_name']
                profile['received_replies'] = self.reply_analysis_data['incoming_replies'].get(screen_name, 0)
                profiles_data.append(profile)

            df = pd.DataFrame(profiles_data)

            # 最近のツイートをまとめる
            tweets_data = []
            for user_data in results:
                profile = user_data['profile']
                for i, tweet in enumerate(user_data.get('recent_tweets', []), 1):
                    if hasattr(tweet, 'text'):
                        tweets_data.append({
                            'screen_name': profile['screen_name'],
                            f'tweet_{i}': tweet.text
                        })

            tweets_df = pd.DataFrame(tweets_data)

            # Excelファイルに保存
            with pd.ExcelWriter(filename) as writer:
                df.to_excel(writer, sheet_name='Profiles', index=False)
                tweets_df.to_excel(writer, sheet_name='Recent_Tweets', index=False)

                # 新機能: リプライ分析シートの追加
                reply_analysis_df = pd.DataFrame({
                    'user': list(self.reply_analysis_data['incoming_replies'].keys()),
                    'received_replies': list(self.reply_analysis_data['incoming_replies'].values())
                })
                reply_analysis_df.to_excel(writer, sheet_name='Reply_Analysis', index=False)

            print(f"Excelファイルを保存しました: {filename}")
        except Exception as e:
            print(f"Excelファイルの保存に失敗: {e}")

In [8]:
# 必要なライブラリのインポート
import os
import json
import pandas as pd
from datetime import datetime
from twikit import Client
import asyncio
from collections import Counter
from google.colab import drive, files, output

# メイン実行関数
async def run_analysis():
    try:
        # アナライザーのインスタンス化
        analyzer = TwitterProfileAnalyzer()

        # セットアップの実行
        if not await analyzer.setup():
            print("セットアップに失敗しました。クッキーファイルを確認してください。")
            return

        # ユーザー入力の取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ").strip()
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析中...")

        # リプライの分析実行
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("リプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの分析
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果の保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

    except Exception as e:
        print(f"分析中にエラーが発生しました: {e}")

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 非同期関数の実行
await run_analysis()

クッキー情報を読み込みました
分析対象のユーザー名を入力してください（@を除く）: wad0427
最小リプライ数を入力してください（例: 3）: 3
分析するツイート数を入力してください（例: 200）: 200

wad0427のリプライを分析中...
リプライ分析エラー: 'NoneType' object has no attribute 'split'
リプライが見つかりませんでした。


In [ ]:
# 必要なライブラリのインポート
import os
import json
import pandas as pd
from datetime import datetime
from twikit import Client
import asyncio
from collections import Counter
from google.colab import drive, output

# TwitterプロフィールとリプライKPI分析用のクラス
class TwitterProfileAnalyzer:
    def __init__(self):
        """
        Twitterプロフィール分析クラスの初期化
        """
        self.client = None
        self.reply_analysis_data = {}

    async def setup(self) -> bool:
        """
        Twitter APIクライアントのセットアップ
        """
        try:
            self.client = Client()

            # クッキーファイルの読み込み（既に存在している前提）
            if os.path.exists(f"{COOKIE_DIR}/cookie_edit.json"):
                with open(f"{COOKIE_DIR}/cookie_edit.json", 'r', encoding='utf-8') as f:
                    self.client.set_cookies(json.load(f))
                return True
            return False
        except Exception as e:
            print(f"セットアップエラー: {e}")
            return False

    async def get_user_tweets(self, user, count=200):
        """
        指定ユーザーの最近のツイートを取得
        """
        try:
            tweets_result = await user.get_tweets('Tweets', count=count)
            return [tweet for tweet in tweets_result if hasattr(tweet, 'text')][:3]
        except Exception as e:
            print(f"ツイート取得エラー: {e}")
            return []

    async def analyze_user_replies(self, target_user: str, tweets_to_analyze: int) -> tuple:
        """
        ユーザーのリプライ分析を実行

        Args:
            target_user: 分析対象のユーザー名
            tweets_to_analyze: 分析対象のツイート数

        Returns:
            tuple: (リプライカウンター, リプライユーザー情報)
        """
        try:
            # ユーザー情報の取得
            user = await self.client.get_user_by_screen_name(target_user)
            if not user:
                print(f"ユーザー {target_user} が見つかりません")
                return Counter(), {}

            # ツイート取得と分析
            tweet_results = await user.get_tweets('Tweets', count=tweets_to_analyze)
            if not tweet_results:
                print("ツイートが取得できません")
                return Counter(), {}

            reply_counter = Counter()
            reply_users = {}
            replied_by_counter = Counter()

            # ツイートごとの解析
            for tweet in tweet_results:
                try:
                    # リプライ情報の解析
                    if hasattr(tweet, 'reply_to') and tweet.reply_to:
                        for reply_tweet in tweet.reply_to:
                            if hasattr(reply_tweet, 'user') and hasattr(reply_tweet.user, 'screen_name'):
                                screen_name = reply_tweet.user.screen_name
                                reply_counter[screen_name] += 1
                                reply_users[screen_name] = reply_tweet.user

                    # 受信リプライの解析
                    if hasattr(tweet, 'reply_count') and tweet.reply_count > 0:
                        try:
                            tweet_detail = await self.client.get_tweet_by_id(tweet.id)
                            if hasattr(tweet_detail, 'replies'):
                                for reply in tweet_detail.replies:
                                    if hasattr(reply.user, 'screen_name'):
                                        replied_by_counter[reply.user.screen_name] += 1
                        except Exception:
                            continue

                except Exception as e:
                    print(f"ツイート解析中のエラー: {e}")
                    continue

            # リプライ分析データの保存
            self.reply_analysis_data = {
                'outgoing_replies': dict(reply_counter),
                'incoming_replies': dict(replied_by_counter)
            }

            return reply_counter, reply_users

        except Exception as e:
            print(f"リプライ分析エラー: {e}")
            return Counter(), {}

    async def get_user_profile(self, user) -> dict:
        """
        ユーザープロフィール情報の取得と整形
        """
        try:
            if not user:
                return {}

            return {
                'screen_name': getattr(user, 'screen_name', ''),
                'name': getattr(user, 'name', ''),
                'description': getattr(user, 'description', ''),
                'followers_count': getattr(user, 'followers_count', 0),
                'following_count': getattr(user, 'following_count', 0),
                'tweets_count': getattr(user, 'statuses_count', 0),
                'created_at': getattr(user, 'created_at', ''),
                'location': getattr(user, 'location', ''),
                'verified': getattr(user, 'verified', False),
                'profile_image_url': getattr(user, 'profile_image_url', '')
            }
        except Exception as e:
            print(f"プロフィール情報取得エラー: {e}")
            return {}

    def save_results(self, results, target_user):
        """
        分析結果をJSONファイルとして保存
        """
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{RESULTS_DIR}/{target_user}_analysis_{timestamp}.json"

        save_data = {
            'user_profiles': results,
            'reply_analysis': self.reply_analysis_data
        }

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(save_data, f, ensure_ascii=False, indent=2)
            print(f"結果を保存しました: {filename}")
        except Exception as e:
            print(f"結果の保存に失敗: {e}")

    def save_to_excel(self, results, target_user):
        """
        分析結果をExcelファイルとして保存
        """
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{RESULTS_DIR}/{target_user}_analysis_{timestamp}.xlsx"

        try:
            # プロフィール情報のDataFrame作成
            profiles_data = []
            for user_data in results:
                profile = user_data['profile']
                profile['reply_count'] = user_data['reply_count']
                screen_name = profile['screen_name']
                profile['received_replies'] = self.reply_analysis_data['incoming_replies'].get(screen_name, 0)
                profiles_data.append(profile)

            df = pd.DataFrame(profiles_data)

            # 最近のツイートをまとめる
            tweets_data = []
            for user_data in results:
                profile = user_data['profile']
                for i, tweet in enumerate(user_data.get('recent_tweets', []), 1):
                    if hasattr(tweet, 'text'):
                        tweets_data.append({
                            'screen_name': profile['screen_name'],
                            f'tweet_{i}': tweet.text
                        })

            tweets_df = pd.DataFrame(tweets_data)

            # Excelファイルに保存
            with pd.ExcelWriter(filename) as writer:
                df.to_excel(writer, sheet_name='Profiles', index=False)
                tweets_df.to_excel(writer, sheet_name='Recent_Tweets', index=False)

                # リプライ分析シート
                reply_analysis_df = pd.DataFrame({
                    'user': list(self.reply_analysis_data['incoming_replies'].keys()),
                    'received_replies': list(self.reply_analysis_data['incoming_replies'].values())
                })
                reply_analysis_df.to_excel(writer, sheet_name='Reply_Analysis', index=False)

            print(f"Excelファイルを保存しました: {filename}")
        except Exception as e:
            print(f"Excelファイルの保存に失敗: {e}")

# メイン実行関数
async def run_analysis():
    try:
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            print("セットアップに失敗しました")
            return

        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ").strip()
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析中...")

        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("リプライが見つかりませんでした。")
            return

        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

    except Exception as e:
        print(f"分析中にエラーが発生しました: {e}")

# Google Colab用の実行コード
output.enable_custom_widget_manager()
await run_analysis()

分析対象のユーザー名を入力してください（@を除く）: keitaro_aigc
最小リプライ数を入力してください（例: 3）: 1
分析するツイート数を入力してください（例: 200）: 200

keitaro_aigcのリプライを分析中...
リプライ分析エラー: 'NoneType' object has no attribute 'split'
リプライが見つかりませんでした。


12/22改良後

In [20]:
async def run_analysis():
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            return

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # リプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 分析の実行
await run_analysis()

2024-12-21 17:16:26,272 - ERROR - cookies.jsonが見つかりません
2024-12-21 17:16:26,272 - ERROR - cookies.jsonが見つかりません
2024-12-21 17:16:26,272 - ERROR - cookies.jsonが見つかりません
2024-12-21 17:16:26,272 - ERROR - cookies.jsonが見つかりません
2024-12-21 17:16:26,272 - ERROR - cookies.jsonが見つかりません
2024-12-21 17:16:26,272 - ERROR - cookies.jsonが見つかりません
ERROR:TwitterAnalyzer:cookies.jsonが見つかりません


12/22改良中

In [28]:
async def run_analysis():
        analyzer = TwitterProfileAnalyzer()

        if not await analyzer.setup():
            return

        # ユーザー入力を対話的に取得
        target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
        min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
        tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

        print(f"\n{target_user}のリプライを分析します...")
        print(f"- 分析対象ツイート数: {tweets_to_analyze}")
        print(f"- 最小リプライ数: {min_replies}")

        # リプライを分析
        reply_counter, reply_users = await analyzer.analyze_user_replies(target_user, tweets_to_analyze)

        if not reply_counter:
            print("\nリプライが見つかりませんでした。")
            return

        # 頻繁にリプライしているユーザーの情報を収集
        frequent_repliers_data = []
        print("\nリプライの多いユーザーの情報を収集中...")

        for screen_name, reply_count in reply_counter.most_common():
            if reply_count >= min_replies and screen_name in reply_users:
                try:
                    user = reply_users[screen_name]
                    profile_data = await analyzer.get_user_profile(user)
                    tweets = await analyzer.get_user_tweets(user, count=3)

                    if profile_data:
                        user_data = {
                            'profile': profile_data,
                            'reply_count': reply_count,
                            'recent_tweets': tweets
                        }
                        frequent_repliers_data.append(user_data)
                        print(f"@{screen_name}の情報を取得しました（リプライ数: {reply_count}）")
                except Exception as e:
                    print(f"@{screen_name}の情報取得に失敗: {e}")
                    continue

        # 結果を保存
        if frequent_repliers_data:
            analyzer.save_results(frequent_repliers_data, target_user)
            analyzer.save_to_excel(frequent_repliers_data, target_user)
            print("\n分析が完了しました！")
        else:
            print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の非同期

# Google Colab用の実行コード
output.enable_custom_widget_manager()

# 分析の実行
await run_analysis()

認証に成功しました！
分析対象のユーザー名を入力してください（@を除く）: sora19ai
最小リプライ数を入力してください（例: 3）: 3
分析するツイート数を入力してください（例: 200）: 400

sora19aiのリプライを分析します...
- 分析対象ツイート数: 400
- 最小リプライ数: 3
sora19aiのツイートを分析中...
ツイートを取得中...
20件のツイートを分析済み
ユーザー 1108k4xi の情報を取得しました
ユーザー solt152152 の情報を取得しました
40件のツイートを分析済み
60件のツイートを分析済み
80件のツイートを分析済み
100件のツイートを分析済み
120件のツイートを分析済み
140件のツイートを分析済み
ユーザー sama の情報を取得しました
160件のツイートを分析済み
180件のツイートを分析済み
200件のツイートを分析済み
ユーザー 3_nipa の情報を取得しました
220件のツイートを分析済み
240件のツイートを分析済み
ユーザー akira_papa_IT の情報を取得しました
ユーザー taiyo_ai_gakuse の情報を取得しました
ユーザー yoshi8__ の情報を取得しました
ユーザー yoshihiko_k の情報を取得しました
ユーザー keitaro_aigc の情報を取得しました
260件のツイートを分析済み
280件のツイートを分析済み
300件のツイートを分析済み
320件のツイートを分析済み
340件のツイートを分析済み
ユーザー bunkaich の情報を取得しました
360件のツイートを分析済み
380件のツイートを分析済み
ユーザー D_Treasure0712 の情報を取得しました
400件のツイートを分析済み

分析完了: 401件のツイートを処理
リプライ先ユーザー数: 11人

リプライの多いユーザーの情報を収集中...

分析対象となるユーザーが見つかりませんでした。


12/22 夜　リプライ３回以上の機能を検証

In [32]:
from collections import Counter
from twikit.client import Client
import asyncio
import json
import pandas as pd
from datetime import datetime

class TwitterProfileAnalyzer:
    def __init__(self):
        # クライアントの初期化
        self.client = Client()
        self.auth_info = None

    async def setup(self):
        """認証情報の設定と接続テスト"""
        try:
            # 認証情報の読み込み
            with open('auth_info.json', 'r') as f:
                self.auth_info = json.load(f)

            # ログイン
            await self.client.login(
                auth_info_1=self.auth_info['username'],
                auth_info_2=self.auth_info['email'],
                password=self.auth_info['password']
            )
            return True
        except Exception as e:
            print(f"セットアップエラー: {e}")
            return False

    async def analyze_user_replies(self, target_user, tweets_to_analyze):
        """ユーザーのリプライを分析"""
        try:
            # ターゲットユーザーの情報を取得
            target_user_obj = await self.client.get_user_by_screen_name(target_user)
            if not target_user_obj:
                return None, None

            # リプライカウンターの初期化
            reply_counter = Counter()
            reply_users = {}

            # 相手からの返信カウンターの初期化
            replies_received_counter = Counter()

            # ユーザーのツイートを取得
            tweets = await self.client.get_user_tweets(
                target_user_obj.id,
                tweet_type='Tweets',
                count=tweets_to_analyze
            )

            print("ツイートを分析中...")
            for tweet in tweets:
                # リプライを分析
                if tweet.in_reply_to:
                    try:
                        replied_tweet = await self.client.get_tweet_by_id(tweet.in_reply_to)
                        if replied_tweet and replied_tweet.user:
                            screen_name = replied_tweet.user.screen_name
                            reply_counter[screen_name] += 1
                            reply_users[screen_name] = replied_tweet.user
                    except Exception as e:
                        continue

                # そのツイートへの返信を取得
                try:
                    replies = await self.client.get_tweet_replies(tweet.id)
                    if replies:
                        for reply in replies:
                            if reply.user:
                                replies_received_counter[reply.user.screen_name] += 1
                except Exception as e:
                    continue

            return reply_counter, reply_users, replies_received_counter
        except Exception as e:
            print(f"分析エラー: {e}")
            return None, None, None

    async def get_user_profile(self, user):
        """ユーザープロフィール情報の取得"""
        try:
            return {
                'screen_name': user.screen_name,
                'name': user.name,
                'description': user.description,
                'followers_count': user.followers_count,
                'following_count': user.following_count,
                'tweets_count': user.statuses_count,
                'created_at': user.created_at,
            }
        except Exception as e:
            print(f"プロフィール取得エラー: {e}")
            return None

    async def get_user_tweets(self, user, count=3):
        """ユーザーの最近のツイートを取得"""
        try:
            tweets = []
            user_tweets = await self.client.get_user_tweets(user.id, tweet_type='Tweets', count=count)
            for tweet in user_tweets:
                tweets.append({
                    'text': tweet.text,
                    'created_at': tweet.created_at,
                    'retweet_count': tweet.retweet_count,
                    'favorite_count': tweet.favorite_count
                })
            return tweets
        except Exception as e:
            print(f"ツイート取得エラー: {e}")
            return []

    def save_results(self, data, target_user):
        """結果をJSONファイルに保存"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{target_user}_analysis_{timestamp}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"結果を{filename}に保存しました。")

    def save_to_excel(self, data, target_user):
        """結果をExcelファイルに保存"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{target_user}_analysis_{timestamp}.xlsx"

        # データフレームの作成
        rows = []
        for user_data in data:
            profile = user_data['profile']
            row = {
                'ユーザー名': f"@{profile['screen_name']}",
                '表示名': profile['name'],
                'プロフィール': profile['description'],
                'フォロワー数': profile['followers_count'],
                'フォロー数': profile['following_count'],
                'ツイート数': profile['tweets_count'],
                'アカウント作成日': profile['created_at'],
                '送信リプライ数': user_data['sent_reply_count'],
                '受信リプライ数': user_data['received_reply_count']
            }
            rows.append(row)

        df = pd.DataFrame(rows)
        df.to_excel(filename, index=False)
        print(f"結果を{filename}に保存しました。")

# メイン実行関数
async def run_analysis():
    analyzer = TwitterProfileAnalyzer()

    if not await analyzer.setup():
        return

    # ユーザー入力を対話的に取得
    target_user = input("分析対象のユーザー名を入力してください（@を除く）: ")
    min_replies = int(input("最小リプライ数を入力してください（例: 3）: "))
    tweets_to_analyze = int(input("分析するツイート数を入力してください（例: 200）: "))

    print(f"\n{target_user}のリプライを分析します...")
    print(f"- 分析対象ツイート数: {tweets_to_analyze}")
    print(f"- 最小リプライ数: {min_replies}")

    # リプライを分析
    reply_counter, reply_users, replies_received_counter = await analyzer.analyze_user_replies(
        target_user, tweets_to_analyze)

    if not reply_counter:
        print("\nリプライが見つかりませんでした。")
        return

    # 頻繁にリプライしているユーザーの情報を収集
    frequent_repliers_data = []
    print("\nリプライの多いユーザーの情報を収集中...")

    for screen_name, sent_reply_count in reply_counter.most_common():
        received_reply_count = replies_received_counter[screen_name]
        total_interactions = sent_reply_count + received_reply_count

        if total_interactions >= min_replies and screen_name in reply_users:
            try:
                user = reply_users[screen_name]
                profile_data = await analyzer.get_user_profile(user)
                tweets = await analyzer.get_user_tweets(user, count=3)

                if profile_data:
                    user_data = {
                        'profile': profile_data,
                        'sent_reply_count': sent_reply_count,
                        'received_reply_count': received_reply_count,
                        'recent_tweets': tweets
                    }
                    frequent_repliers_data.append(user_data)
                    print(f"@{screen_name}の情報を取得しました（送信リプライ数: {sent_reply_count}, 受信リプライ数: {received_reply_count}）")
            except Exception as e:
                print(f"@{screen_name}の情報取得に失敗: {e}")
                continue

    # 結果を保存
    if frequent_repliers_data:
        analyzer.save_results(frequent_repliers_data, target_user)
        analyzer.save_to_excel(frequent_repliers_data, target_user)
        print("\n分析が完了しました！")
    else:
        print("\n分析対象となるユーザーが見つかりませんでした。")

# Google Colab用の実行コード
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()
    await run_analysis()
else:
    asyncio.run(run_analysis())


ImportError: cannot import name 'Client' from 'twikit.client' (unknown location)

# キーワード検索

キーワード検索　初期設定のセル

In [ ]:
# 必要なライブラリのインストール
!pip install twikit pandas openpyxl

# 必要なインポート
from twikit import Client
import json
import os
from datetime import datetime
import asyncio
import pandas as pd
from google.colab import drive
from google.colab import files

# Google Driveのマウント
drive.mount('/content/drive')

# ディレクトリ設定
WORK_DIR = '/content/drive/MyDrive/Twitter_Analysis'
os.makedirs(WORK_DIR, exist_ok=True)

COOKIE_DIR = f"{WORK_DIR}/twitter_json"
os.makedirs(COOKIE_DIR, exist_ok=True)

RESULTS_DIR = f"{WORK_DIR}/keyword_search_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print("ディレクトリ設定完了！")
print(f"作業ディレクトリ: {WORK_DIR}")
print(f"クッキー保存先: {COOKIE_DIR}")
print(f"結果保存先: {RESULTS_DIR}")

# クッキーファイルのアップロード
def upload_cookie_file():
    print("cookie_edit.jsonファイルをアップロードしてください")
    uploaded = files.upload()

    if 'cookie_edit.json' in uploaded:
        with open(f"{COOKIE_DIR}/cookie_edit.json", 'wb') as f:
            f.write(uploaded['cookie_edit.json'])
        print("クッキーファイルを保存しました")
        return True
    else:
        print("cookie_edit.jsonファイルがアップロードされませんでした")
        return False

upload_cookie_file()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ディレクトリ設定完了！
作業ディレクトリ: /content/drive/MyDrive/Twitter_Analysis
クッキー保存先: /content/drive/MyDrive/Twitter_Analysis/twitter_json
結果保存先: /content/drive/MyDrive/Twitter_Analysis/keyword_search_results
cookie_edit.jsonファイルをアップロードしてください


Saving cookie_edit.json to cookie_edit (1).json
cookie_edit.jsonファイルがアップロードされませんでした


False

キーワード検索　クラス定義セル

In [ ]:
class TwitterKeywordAnalyzer:
    def __init__(self):
        self.client = Client(language='en-US')
        self.cookie_path = f"{COOKIE_DIR}/cookie_edit.json"
        self.results_dir = RESULTS_DIR

    async def setup(self):
        try:
            with open(self.cookie_path, 'r', encoding='utf-8') as file:
                cookies = json.load(file)
            self.client.set_cookies(cookies)
            print("認証に成功しました！")
            return True
        except Exception as e:
            print(f"認証エラー: {e}")
            return False

    async def search_with_keyword(self, keyword, count=20, sort_by='latest'):
        """キーワードを含むツイートを検索し、関連情報を取得する"""
        try:
            search_results = []
            print(f"'{keyword}' に関連する情報を検索中...(並び順: {sort_by})")

            product_type = 'Latest' if sort_by == 'latest' else 'Top'
            results = await self.client.search_tweet(
                query=keyword,
                product=product_type,
                count=min(count, 20)
            )

            for tweet in results:
                user_data = {
                    'user_id': tweet.user.id,
                    'name': tweet.user.name,
                    'screen_name': tweet.user.screen_name,
                    'profile_description': tweet.user.description,
                    'profile_url': f"https://twitter.com/{tweet.user.screen_name}",
                    'followers_count': tweet.user.followers_count,
                    'following_count': tweet.user.following_count,
                    'profile_image_url': tweet.user.profile_image_url,
                    'location': tweet.user.location
                }

                tweet_data = {
                    'tweet_id': tweet.id,
                    'tweet_url': f"https://twitter.com/{tweet.user.screen_name}/status/{tweet.id}",
                    'text': tweet.text,
                    'created_at': tweet.created_at,
                    'retweet_count': tweet.retweet_count,
                    'like_count': tweet.favorite_count,
                    'reply_count': tweet.reply_count if hasattr(tweet, 'reply_count') else 0,
                    'is_retweet': bool(tweet.retweeted_tweet),
                    'is_quote': tweet.is_quote_status,
                    'language': tweet.lang
                }

                keyword_locations = []
                if keyword.lower() in tweet.text.lower():
                    keyword_locations.append('tweet_text')
                if keyword.lower() in tweet.user.description.lower():
                    keyword_locations.append('profile_description')
                if keyword.lower() in tweet.user.name.lower():
                    keyword_locations.append('user_name')
                if keyword.lower() in tweet.user.screen_name.lower():
                    keyword_locations.append('screen_name')

                search_results.append({
                    'user': user_data,
                    'tweet': tweet_data,
                    'keyword_locations': keyword_locations
                })

            if sort_by == 'likes':
                search_results.sort(key=lambda x: x['tweet']['like_count'], reverse=True)

            return search_results

        except Exception as e:
            print(f"検索エラー: {e}")
            return []

    # save_results と save_to_excel メソッドはそのまま
    def save_results(self, results, filename_prefix):
        """検索結果をJSONファイルとして保存する"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.results_dir}/{filename_prefix}_{timestamp}.json"

        try:
            with open(filename, 'w', encoding='utf-8') as file:
                json.dump(results, file, ensure_ascii=False, indent=2, default=str)
            print(f"\n結果を保存しました: {filename}")
        except Exception as e:
            print(f"結果の保存中にエラーが発生しました: {e}")

    def save_to_excel(self, results, keyword, sort_by):
        """検索結果をExcelファイルとして保存"""
        try:
            # 結果をDataFrame用に整形
            rows = []
            for result in results:
                user = result['user']
                tweet = result['tweet']
                locations = result['keyword_locations']

                row = {
                    '検索日時': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    '検索キーワード': keyword,
                    '並び順': sort_by,
                    '投稿日時': tweet['created_at'],
                    'アカウント名': user['name'],
                    'ユーザーID': f"@{user['screen_name']}",
                    'プロフィール文': user['profile_description'],
                    'フォロワー数': user['followers_count'],
                    'フォロー数': user['following_count'],
                    'ツイート本文': tweet['text'],
                    'いいね数': tweet['like_count'],
                    'リツイート数': tweet['retweet_count'],
                    'リプライ数': tweet['reply_count'],
                    'ツイートURL': tweet['tweet_url'],
                    'アカウントURL': user['profile_url'],
                    'キーワード出現場所': ', '.join(locations),
                    '場所': user['location'] if user['location'] else '',
                    '言語': tweet['language']
                }
                rows.append(row)

            # DataFrameを作成
            df = pd.DataFrame(rows)

            # Excelファイル名を生成
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            excel_file = f"{self.results_dir}/Twitter検索結果_{keyword}_{sort_by}_{timestamp}.xlsx"

            # Excelファイルとして保存
            with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='検索結果', index=False)

                # 列幅の自動調整
                worksheet = writer.sheets['検索結果']
                for idx, col in enumerate(df.columns):
                    max_length = max(
                        df[col].astype(str).apply(len).max(),
                        len(str(col))
                    )
                    # 最大幅を50文字に制限
                    adjusted_width = min(max_length + 2, 50)
                    worksheet.column_dimensions[chr(65 + idx)].width = adjusted_width

            print(f"\nExcelファイルを保存しました: {excel_file}")
            return excel_file

        except Exception as e:
            print(f"Excelファイルの保存中にエラーが発生しました: {e}")
            return None

キーワード検索　実行セル

In [ ]:
async def run_search():
    analyzer = TwitterKeywordAnalyzer()

    if not await analyzer.setup():
        return

    # 検索設定を対話的に取得
    keyword = input("検索キーワードを入力してください: ")
    count = int(input("取得する結果の数を入力してください（最大20）: "))

    print("\n検索オプション:")
    print("1: 新しい順")
    print("2: 人気順")
    print("3: いいね数順")

    try:
        option = int(input("検索オプションを選択してください (1-3): "))
        sort_by = {
            1: 'latest',
            2: 'top',
            3: 'likes'
        }.get(option, 'latest')
    except ValueError:
        print("無効な入力です。デフォルトの'新しい順'で検索します。")
        sort_by = 'latest'

    results = await analyzer.search_with_keyword(keyword, count, sort_by)

    if not results:
        print("検索結果が見つかりませんでした。")
        return

    sort_type = {
        'latest': '新しい順',
        'top': '人気順',
        'likes': 'いいね数順'
    }[sort_by]

    print(f"\n検索結果 ({len(results)} 件) - {sort_type}:")
    for result in results:
        print("\n" + "="*50)
        user = result['user']
        tweet = result['tweet']

        print(f"投稿日時: {tweet['created_at']}")
        print(f"いいね数: {tweet['like_count']}")
        print(f"\nユーザー情報: {user['name']} (@{user['screen_name']})")
        print(f"ツイート: {tweet['text']}")
        print(f"URL: {tweet['tweet_url']}")

    if results:
        analyzer.save_results(results, f"{keyword}_{sort_by}")
        analyzer.save_to_excel(results, keyword, sort_type)

# Google Colab用の実行
from google.colab import output
output.enable_custom_widget_manager()

# 分析実行
await run_search()

認証に成功しました！
検索キーワードを入力してください: dify
取得する結果の数を入力してください（最大20）: 20

検索オプション:
1: 新しい順
2: 人気順
3: いいね数順
検索オプションを選択してください (1-3): 2
'dify' に関連する情報を検索中...(並び順: top)

検索結果 (19 件) - 人気順:

投稿日時: Wed Dec 11 03:06:26 +0000 2024
いいね数: 6

ユーザー情報: katoshin (@katoshin_ii)
ツイート: 要件定義を対話型で実現するDifyのチャットフローがなかなか大作になってしまった。 https://t.co/JKJNpFFn4A
URL: https://twitter.com/katoshin_ii/status/1866680917722632626

投稿日時: Sat Dec 07 07:53:30 +0000 2024
いいね数: 171

ユーザー情報: Tom | ドバイで生成AIやってる人 (@0x__tom)
ツイート: Difyを使えば、「食事や体重の記録」から「記録に基づいたアドバイスの生成」までを一瞬で自動化できる。写真をアップするだけで、カロリー計算や体重をスプシに記録し、AIがデータを分析してパーソナライズされたアドバイスを提案。面倒な記録の手間を省き、健康管理をサポートしてくれます。 https://t.co/3YkdhDSbAb
URL: https://twitter.com/0x__tom/status/1865303606683164768

投稿日時: Wed Dec 11 16:35:55 +0000 2024
いいね数: 5

ユーザー情報: 森本洋平@すもちゃん| 動画×AI・Dify開発 (@sumo_wonderful)
ツイート: Soraの記念すべき初体験AI生成は「sumo wrestler」と入力してストーリーもAIに自動作成してもらった。

いくらすごいAI動画生成ツールでも"相撲"の学習量と知名度が少なすぎるんだろうね。
AIツールを最初に試す時、相撲関連のプロンプト当てると大体そのAIツールの限界が見える。 https://t.co/4WiuQOHgNC
URL: https://twitt